In [8]:
from dotenv import load_dotenv
import os
from pathlib import Path
import psycopg2 as pg


# Load environment variables
ENV_PATH = "./.env"
load_dotenv(ENV_PATH)

False

## Setup

In [9]:
from pyhive import hive
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, when
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import shutil

db_host = os.getenv("POSTGRES_HOST","localhost")
db_port = os.getenv("POSTGRES_PORT","1000")
db_user = os.getenv("POSTGRES_USER","postgres")
db_password = os.getenv("POSTGRES_PASSWORD","postgres")
db_database = os.getenv("POSTGRES_DATABASE","test")

# Create a connection to the PostgreSQL database
pg_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}')


## etl restaurant detail to hive

In [41]:
etl_restaurant_statement = """
    with 
    lastest_dt_restaurant as (
        SELECT restaurant_id, max(TO_CHAR(order_created_timestamp,'YYYYMMDD')) as dt 
        FROM public.order_detail
        GROUP BY restaurant_id
    )

    select rd.*, ldr.dt
    from public.restaurant_detail rd
    left join lastest_dt_restaurant ldr on rd.id = ldr.restaurant_id
"""
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .enableHiveSupport() \
    .getOrCreate()

with pg_engine.connect() as pg_conn:
    df_restaurant = pd.read_sql_query(etl_restaurant_statement, pg_conn)
    spark_df_restaurant = spark.createDataFrame(df_restaurant)
    spark_df_restaurant.write.mode("overwrite").format("parquet").partitionBy("dt").saveAsTable("restaurant_detail")

# print(spark.conf.get("spark.sql.warehouse.dir"))


23/12/11 06:22:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/12/11 06:22:21 ERROR FileOutputCommitter: Mkdirs failed to create file:/opt/hive/data/warehouse/restaurant_detail/_temporary/0
23/12/11 06:22:21 ERROR Utils: Aborting task
java.io.IOException: Mkdirs failed to create file:/opt/hive/data/warehouse/restaurant_detail/_temporary/0/_temporary/attempt_20231211062221857091061058753120_0004_m_000011_267/dt=20190524 (exists=false, cwd=file:/home/best/airflow-hive-spark-lw/playground/notebooks)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetF

Py4JJavaError: An error occurred while calling o276.saveAsTable.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 45 in stage 4.0 failed 1 times, most recent failure: Lost task 45.0 in stage 4.0 (TID 301) (super01.rnd executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/opt/hive/data/warehouse/restaurant_detail.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.co

In [ ]:
    df_order_detail = pd.read_sql_query("select * from order_detail", pg_conn)
    spark_df_order = spark.createDataFrame(df_order_detail) 
    spark_df_order = spark_df_order.withColumn("dt", date_format(col("order_created_timestamp"),"yyyymmdd"))
    spark_df_order.write.mode("overwrite").format("parquet").partitionBy("dt").saveAsTable("order_detail")

In [ ]:
spark_df_order_new = spark.sql("SELECT * FROM order_detail")
spark_df_order_new = spark_df_order_new.withColumn("discount_no_null", col("discount")).fillna(0)
spark_df_order_new.write.mode("overwrite").saveAsTable("__order_detail_new__")

In [ ]:
spark_df_order_new.printSchema()

In [ ]:

spark_df_restaurant_new = spark.sql("SELECT * FROM restaurant_detail")
spark_df_restaurant_new = spark_df_restaurant_new.withColumn(
    "cooking_bin", 
    when(col("esimated_cooking_time").between(1,40),1)
    .when(col("esimated_cooking_time").between(41,80),2)
    .when(col("esimated_cooking_time").between(81,120),3)
    .when(col("esimated_cooking_time")>120,4)
    .otherwise(0)
)
spark_df_restaurant_new.write.mode("overwrite").saveAsTable("__restaurant_detail_new__")

In [ ]:
spark_df_restaurant_new.show()

In [ ]:
spark_df_restaurant.printSchema()

In [ ]:
spark_df_order_new.printSchema()

In [ ]:
statement = """
    SELECT res.category, AVG(order.discount_no_null) AS avg_discount
    FROM __order_detail_new__ AS order
    LEFT JOIN restaurant_detail AS res ON order.restaurant_id = res.id
    GROUP BY res.category
"""

df = spark.sql(statement)
df.write.csv("../sql_requirements/avg_discount_by_category", header=True)

In [ ]:
statement = """
    SELECT cooking_bin, COUNT(*) as row_count
    FROM __restaurant_detail_new__
    GROUP BY cooking_bin
"""

df = spark.sql(statement)
df.write.csv("../sql_requirements/row_count_by_cooking_bin", header=True)

In [ ]:
df.show()

In [35]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName("test") \
            .config("spark.driver.memory", "4g") \
            .config("spark.executor.memory", "4g") \
            .enableHiveSupport() \
            .getOrCreate()

print(spark.conf.get("hive.metastore.warehouse.dir"))


/opt/hive/data/warehouse
